In [1]:

from transformers import(
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
)
from tqdm import tqdm
from datasets import Dataset
import os
import json
from sentence_transformers import SentenceTransformer, util
import torch
import csv
import math
import random
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
import torch
from rouge import Rouge
from transformers import AutoTokenizer
from summarizer.sbert import SBertSummarizer
from summarizer import Summarizer
from transformers import *
import nltk
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader
import logging
from sentence_transformers import SentenceTransformer, util
import torch
import csv
import re
from langchain_text_splitters import NLTKTextSplitter

/home/zhiweny2/utils/miniconda3/envs/multi-tagger/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zhiweny2/utils/miniconda3/envs/multi-tagger/lib/python3.8/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/home/zhiweny2/utils/miniconda3/envs/multi-tagger/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
import random
import numpy as np
# set random seed
random_seed = 42
random.seed(random_seed)

In [3]:
def load_data(dataset, datatype):

    data_folder = '/home/zhiweny2/chatbotai/jerome/BioNLP/biolaysumm2024_data'
    data_path = os.path.join(data_folder, f'{dataset}_{datatype}.jsonl')
    lay_sum = []
    article =[]
    keyword = []
    headings = []
    id = []
    file = open(data_path, 'r')
    for line in (file.readlines()):
        dic = json.loads(line)
        article.append(dic['article'])
        keyword.append(dic['keywords'])
        headings.append(dic['headings'])
        id.append(dic['id'])
        lay_sum.append(dic['lay_summary'])
    
    return article, lay_sum, keyword, headings, id

def load_test_data(dataset, datatype):

    data_folder = '/home/zhiweny2/chatbotai/jerome/BioNLP/biolaysumm2024_data'
    
    data_path = os.path.join(data_folder, f'{dataset}_{datatype}.jsonl')
    article =[]
    keyword = []
    headings = []
    id = []
    file = open(data_path, 'r')
    for line in (file.readlines()):
        dic = json.loads(line)
        article.append(dic['article'])
        keyword.append(dic['keywords'])
        headings.append(dic['headings'])
        id.append(dic['id'])
    
    return article, keyword, headings, id

In [4]:
background = []
objective = []
methods = []
results = []
conclusions = []

with open('/home/zhiweny2/chatbotai/jerome/BioNLP/Structured-Abstracts-Labels-102615.txt', 'r') as file:
    for line in file:
        components = line.strip().split('|')
        title, category, _, _ = components
        if category == 'BACKGROUND':
            background.append(title)
        elif category == 'OBJECTIVE':
            objective.append(title)
        elif category == 'METHODS':
            methods.append(title)
        elif category == 'RESULTS':
            results.append(title)
        elif category == 'CONCLUSIONS':
            conclusions.append(title)
            
background = [item.lower() for item in background]
objective = [item.lower() for item in objective]
methods = [item.lower() for item in methods]
results = [item.lower() for item in results]
conclusions = [item.lower() for item in conclusions]
    
print(background)
print(conclusions)

['abbreviations', 'access to data', 'accme accreditation', 'acknowledgments', 'aetiology', 'anamnesis', 'anamnesis and clinical findings', 'antecedents', 'approach to the problem', 'article title and bibliographic information', 'audience', 'availability', 'availability and supplementary information', 'background', 'background and context', 'background and importance', 'background and introduction', 'background and motivation', 'background and overview', 'background and significance', 'background content', 'background context', 'background data', 'background information', 'background of the study', 'background to the debate', 'background/introduction', 'background/rationale', 'background/significance', 'backgrounds', 'basic remarks', 'basics', 'basis', 'citation', 'clinical trial', 'clinical trial information', 'clinical trial number', 'clinical trial registration', 'clinical trial registration information', 'clinical trial registration number', 'clinical trial registration url', 'clini

In [5]:
### PLOS
# train
plos_article_train, plos_lay_sum_train, plos_keyword_train, plos_headings_train, plos_id_train = load_data('PLOS', 'train')
# val
plos_article_val, plos_lay_sum_val, plos_keyword_val, plos_headings_val, plos_id_val = load_data('PLOS', 'val')
# # test
# plos_article_test, plos_keyword_test, plos_headings_test, plos_id_test = load_test_data('PLOS', 'test')


### eLife
# train
# elife_article_train, elife_lay_sum_train, elife_keyword_train, elife_headings_train, elife_id_train = load_data('eLife', 'train')
# # val
# elife_article_val, elife_lay_sum_val, elife_keyword_val, elife_headings_val, elife_id_val = load_data('eLife', 'val')
# # # test
# elife_article_test, elife_keyword_test, elife_headings_test, elife_id_test = load_test_data('eLife', 'test')

In [ ]:
### chunk articles and lay summs for PLOS 

pattern = r'\s\[.*?\]'
text_splitter = NLTKTextSplitter(chunk_size=600)

### train
new_plos_article_train = []
for s in plos_article_train:
    new_s = s.replace(' . ', '. ')
    new_s = new_s.replace(' , ', ', ')
    new_plos_article_train.append(new_s)
print(len(new_plos_article_train))

chunked_plos_article_train = []
for article in new_plos_article_train:
    texts = text_splitter.split_text(article)
    new_texts = []
    for t in texts:
        t = t.replace('\n\n', ' ')
        ### remove the irrelevant citations and references
        result = re.sub(pattern, "", t)
        result = result.replace(' , ', ', ')
        new_texts.append(result)
    chunked_plos_article_train.append(new_texts)
print(len(chunked_plos_article_train))
    

### val
new_plos_article_val = []
for s in plos_article_val:
    new_s = s.replace(' . ', '. ')
    new_s = new_s.replace(' , ', ', ')
    new_plos_article_val.append(new_s)
print(len(new_plos_article_val))

chunked_plos_article_val = []  # 100 tokens per chunk
for article in new_plos_article_val:
    texts = text_splitter.split_text(article)
    new_texts = []
    for t in texts:
        t = t.replace('\n\n', ' ')
        result = re.sub(pattern, "", t)
        result = result.replace(' , ', ', ')
        new_texts.append(result)
    chunked_plos_article_val.append(new_texts)


### train lay sum
new_plos_lay_sum_train = []
for s in plos_lay_sum_train:
    new_s = s.replace(' . ', '. ')
    new_s = new_s.replace(' , ', ', ')
    new_plos_lay_sum_train.append(new_s)

chunked_plos_lay_sum_train = []
for sum in new_plos_lay_sum_train:
    texts = text_splitter.split_text(sum)
    new_texts = []
    for t in texts:
        t = t.replace('\n\n', ' ')
        new_texts.append(t)
    chunked_plos_lay_sum_train.append(new_texts)

### val lay sum
new_plos_lay_sum_val = []
for s in plos_lay_sum_val:
    new_s = s.replace(' . ', '. ')
    new_s = new_s.replace(' , ', ', ')
    new_plos_lay_sum_val.append(new_s)

chunked_plos_lay_sum_val = []
for sum in new_plos_lay_sum_val:
    texts = text_splitter.split_text(sum)
    new_texts = []
    for t in texts:
        t = t.replace('\n\n', ' ')
        new_texts.append(t)
    chunked_plos_lay_sum_val.append(new_texts)
print(len(chunked_plos_lay_sum_val))

In [ ]:
### chunk articles and lay summs for eLife

# pattern = r'(\(\s(.*)\,\s\d{4}\s\))'
# pattern = r'(\(\s([A-Za-z\s]+ \w*)+\.*\,\s\d{4}.*\s\))'
# pattern = r'(\(\s([A-Za-z]+(\s[a-z]*)*)\.*\,\s\d{4}\s\))|(\(\s([A-Za-z\s]+ \w*)+\.*\,\s\d{4}\s\))'
# (\(\s([A-Za-z]+(\s[a-z]*)*)\.*\,\s\d{4}\s\))|(\(\s([A-Za-z^0-9\s]+ \w*)+\.*\,\s\d{4}\s\))|

pattern = r'(\(\s([^()]*\s\,\s)*[^()]*\s\))'
text_splitter = NLTKTextSplitter(chunk_size=600)

### train
new_elife_article_train = []
for s in elife_article_train:
    new_s = s.replace(' . ', '. ')
    new_s = new_s.replace(' , ', ', ')
    new_elife_article_train.append(new_s)
print(len(new_elife_article_train))

chunked_elife_article_train = []
for article in new_elife_article_train:
    texts = text_splitter.split_text(article)
    new_texts = []
    for t in texts:
        t = t.replace('\n\n', ' ')
        ### remove the irrelevant citations and references
        result = re.sub(pattern, "", t)
        result = result.replace(' , ', ', ')
        new_texts.append(result)
    chunked_elife_article_train.append(new_texts)
print(len(chunked_elife_article_train))
    

### val
new_elife_article_val = []
for s in elife_article_val:
    new_s = s.replace(' . ', '. ')
    new_s = new_s.replace(' , ', ', ')
    new_elife_article_val.append(new_s)
print(len(new_elife_article_val))

chunked_elife_article_val = []
for article in new_elife_article_val:
    texts = text_splitter.split_text(article)
    new_texts = []
    for t in texts:
        t = t.replace('\n\n', ' ')
        ### remove the irrelevant citations and references
        result = re.sub(pattern, "", t)
        result = result.replace(' , ', ', ')
        new_texts.append(result)
    chunked_elife_article_val.append(new_texts)


### train lay sum
new_elife_lay_sum_train = []
for s in elife_lay_sum_train:
    new_s = s.replace(' . ', '. ')
    new_s = new_s.replace(' , ', ', ')
    new_elife_lay_sum_train.append(new_s)

chunked_elife_lay_sum_train = []
for sum in new_elife_lay_sum_train:
    texts = text_splitter.split_text(sum)
    new_texts = []
    for t in texts:
        t = t.replace('\n\n', ' ')
        new_texts.append(t)
    chunked_elife_lay_sum_train.append(new_texts)

### val lay sum
new_elife_lay_sum_val = []
for s in elife_lay_sum_val:
    new_s = s.replace(' . ', '. ')
    new_s = new_s.replace(' , ', ', ')
    new_elife_lay_sum_val.append(new_s)

chunked_elife_lay_sum_val = []
for sum in new_elife_lay_sum_val:
    texts = text_splitter.split_text(sum)
    new_texts = []
    for t in texts:
        t = t.replace('\n\n', ' ')
        new_texts.append(t)
    chunked_elife_lay_sum_val.append(new_texts)
print(len(chunked_elife_lay_sum_val))

In [17]:
from sentence_transformers import SentenceTransformer, util

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer("NeuML/pubmedbert-base-embeddings").to(device)

### val contrastive datasets creation

positive_chunk = []
negative_chunk = []
for lay_sum, doc in zip(chunked_plos_lay_sum_val, chunked_plos_article_val):
    doc_embeddings = model.encode(doc, convert_to_tensor=True)
    lay_embeddings = model.encode(lay_sum, convert_to_tensor=True)
    cosine_scores = util.cos_sim(lay_embeddings, doc_embeddings)
    for i in range(cosine_scores.shape[0]):
        for j in range(cosine_scores.shape[1]):
            if cosine_scores[i][j] >= 0.9: #0.8
                positive_chunk.append((lay_sum[i], doc[j], 1))
            elif cosine_scores[i][j] <= 0.01: #0.01
                negative_chunk.append((lay_sum[i], doc[j], 0))

print("Number of positive sentences:", len(positive_chunk))
print("Number of negative sentences:", len(negative_chunk))

selected_samples = random.sample(negative_chunk, 16210)

with open('/home/zhiweny2/chatbotai/jerome/BioNLP/plos_val_sentence_level_positive_negative_pairs.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['lay_sum', 'sentence', 'label'])
    for row in positive_chunk:
        writer.writerow(row)
    for row in negative_chunk:
        writer.writerow(row)

loading configuration file config.json from cache at /home/zhiweny2/.cache/huggingface/hub/models--NeuML--pubmedbert-base-embeddings/snapshots/ba210f40b1b6d555d675c2d1ed6372e44570fc3c/config.json
Model config BertConfig {
  "_name_or_path": "NeuML/pubmedbert-base-embeddings",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/zhiweny2/.cache/huggingface/hub/models--NeuML--pubmedbert-base-embeddings/snapshot

Number of positive sentences: 1070
Number of negative sentences: 3957


In [21]:
### train contrastive datasets creation
positive_chunk = []
negative_chunk = []

for lay_sum, doc in zip(chunked_plos_lay_sum_train, chunked_plos_article_train):
    doc_embeddings = model.encode(doc, convert_to_tensor=True)
    lay_embeddings = model.encode(lay_sum, convert_to_tensor=True)
    cosine_scores = util.cos_sim(lay_embeddings, doc_embeddings)
    for i in range(cosine_scores.shape[0]):
        for j in range(cosine_scores.shape[1]):
            if cosine_scores[i][j] >= 0.9:
                positive_chunk.append((lay_sum[i], doc[j], 1))
            elif cosine_scores[i][j] <= 0.01:
                negative_chunk.append((lay_sum[i], doc[j], 0))
                
print("Number of positive sentences:", len(positive_chunk))
print("Number of negative sentences:", len(negative_chunk))

selected_samples = random.sample(negative_chunk, 17910)

with open('/home/zhiweny2/chatbotai/jerome/BioNLP/plos_train_sentence_level_positive_negative_pairs.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['lay_sum', 'sentence', 'label'])
    for row in positive_chunk:
        writer.writerow(row)
    for row in selected_samples:
        writer.writerow(row)

Number of positive sentences: 17910
Number of negative sentences: 73604


### Fine-Tune a Embedding Model through Contrastive Loss

In [6]:
train_data = []
with open('/home/zhiweny2/chatbotai/jerome/BioNLP/plos_train_sentence_level_positive_negative_pairs.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader) 
    for row in reader:
        train_data.append(row)
random.shuffle(train_data)

val_data = [] #1824
with open('/home/zhiweny2/chatbotai/jerome/BioNLP/plos_val_sentence_level_positive_negative_pairs.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader) 
    for row in reader:
        val_data.append(row)
random.shuffle(val_data)

print(len(train_data))
print(len(val_data))

35820
2140


In [7]:
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader
import logging

# parameters
train_batch_size = 32
model = SentenceTransformer('NeuML/pubmedbert-base-embeddings').to('cuda:2')
num_epochs = 5
model_save_path = '/home/zhiweny2/chatbotai/jerome/BioNLP/plos_sentence_level_embedding_model'
logging.info("Read train dataset")

train_examples = []
val_examples = []
for row in train_data:
  score = int(row[2])
  train_examples.append(InputExample(texts = [str(row[0]), str(row[1])], label = score))
  
for row in val_data:
  score = int(row[2])
  val_examples.append(InputExample(texts = [str(row[0]), str(row[1])], label = score))

print(len(train_examples))
print(len(val_examples))

train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle = True, batch_size = train_batch_size)
train_loss = losses.ContrastiveLoss(model = model)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(val_examples, name='dev-set')

# Configure the training
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))
evaluation_steps = int(len(train_dataloader) * 1)
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=evaluation_steps,
          warmup_steps=warmup_steps,
          show_progress_bar=True,
          save_best_model = True,
          output_path=model_save_path)

loading configuration file config.json from cache at /home/zhiweny2/.cache/huggingface/hub/models--NeuML--pubmedbert-base-embeddings/snapshots/ba210f40b1b6d555d675c2d1ed6372e44570fc3c/config.json
Model config BertConfig {
  "_name_or_path": "NeuML/pubmedbert-base-embeddings",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/zhiweny2/.cache/huggingface/hub/models--NeuML--pubmedbert-base-embeddings/snapshot

35820
2140


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Configuration saved in /home/zhiweny2/chatbotai/jerome/BioNLP/plos_sentence_level_embedding_model/config.json
Model weights saved in /home/zhiweny2/chatbotai/jerome/BioNLP/plos_sentence_level_embedding_model/pytorch_model.bin
tokenizer config file saved in /home/zhiweny2/chatbotai/jerome/BioNLP/plos_sentence_level_embedding_model/tokenizer_config.json
Special tokens file saved in /home/zhiweny2/chatbotai/jerome/BioNLP/plos_sentence_level_embedding_model/special_tokens_map.json
Epoch:  80%|████████  | 4/5 [41:37<10:23, 623.57s/it]Configuration saved in /home/zhiweny2/chatbotai/jerome/BioNLP/plos_sentence_level_embedding_model/config.json
Model weights saved in /home/zhiweny2/chatbotai/jerome/BioNLP/plos_sentence_level_embedding_model/pytorch_model.bin
tokenizer config file saved in /home/zhiweny2/chatbotai/jerome/BioNLP/plos_sentence_level_embedding_model/tokenizer_config.json
Special tokens file saved in /home/zhiweny2/chatbotai/jerome/BioNLP

In [7]:
model.eval()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [8]:
test_examples = []
labels = []
for row in val_data:
  score = int(row[2])
  labels.append(score)
  test_examples.append(InputExample(texts = [str(row[0]), str(row[1])], label = score))

print(len(test_examples))
# model = SentenceTransformer('/home/zhiweny2/chatbotai/jerome/BioNLP/sentence_level_embedding_model').to('cuda:3')
# train_loss = losses.ContrastiveLoss(model = model)
test_data = SentencesDataset(test_examples, model=model)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=128)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_examples, name='test-set')
evaluator(model)

2140


0.8660255001926743

In [ ]:
### create new extractive datasets
text_splitter = NLTKTextSplitter(chunk_size=600)

custom_config = AutoConfig.from_pretrained('/home/zhiweny2/chatbotai/jerome/BioNLP/sentence_level_embedding_model')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('/home/zhiweny2/chatbotai/jerome/BioNLP/sentence_level_embedding_model')
custom_model = AutoModel.from_pretrained('/home/zhiweny2/chatbotai/jerome/BioNLP/sentence_level_embedding_model', config=custom_config)
model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

### train
final_elife_lay_sum_train = []
new_selected_elife_article_train = []
for lay, article, headings in zip(new_elife_lay_sum_train, new_elife_article_train, elife_headings_train):
    sections = article.split('\n')
    temp_selected_sections = []
    temp_sections = []
    for i, (heading, section) in enumerate(zip(headings, sections)):
        heading = heading.lower()
        if heading in background:
            temp_selected_sections.append(section)
        elif heading in methods:
            temp_sections.append(section)
        elif heading in conclusions:
            temp_sections.append(section)
        elif heading in results:
            temp_sections.append(section)
        elif heading in 'abstract':
            temp_selected_sections.append(section)
        else:
            continue
        
    final_string = '\n'.join(temp_sections)
    final_selected_string = '\n'.join(temp_selected_sections)

    chunked_string = ""
    summarized_sections = ""
    if final_selected_string:
        final_elife_lay_sum_train.append(lay)
    
        if final_string:
            a = text_splitter.split_text(final_string)
            new_texts = []
            for t in a:
                t = t.replace('\n\n', ' ')
                ### remove the irrelevant citations and references
                result = re.sub(pattern, "", t)
                result = result.replace(' , ', ', ')
                new_texts.append(result)
            # doc_func = lambda x: x.page_content
            # docs = list(map(doc_func, a))
            chunked_string = ' '.join(new_texts)

        if chunked_string:
            result = model(chunked_string, num_sentences=50, max_length=256)
            summarized_sections = ''.join(result)
        if summarized_sections:
            final_selected_string = final_selected_string + '\n' + summarized_sections
            new_selected_elife_article_train.append(final_selected_string)
        else:
            new_selected_elife_article_train.append(final_selected_string)
            
print(len(new_selected_elife_article_train))


### val
final_elife_lay_sum_val = []
new_selected_elife_article_val = []
for lay, article, headings in zip(new_elife_lay_sum_val, new_elife_article_val, elife_headings_val):
    sections = article.split('\n')
    temp_selected_sections = []
    temp_sections = []
    for i, (heading, section) in enumerate(zip(headings, sections)):
        heading = heading.lower()
        if heading in background:
            temp_selected_sections.append(section)
        elif heading in methods:
            temp_sections.append(section)
        elif heading in conclusions:
            temp_sections.append(section)
        elif heading in results:
            temp_sections.append(section)
        elif heading in 'abstract':
            temp_selected_sections.append(section)
        else:
            continue
        
    final_string = '\n'.join(temp_sections)
    final_selected_string = '\n'.join(temp_selected_sections)

    chunked_string = ""
    summarized_sections = ""
    if final_selected_string:
        final_elife_lay_sum_val.append(lay)
    
        if final_string:
            a = text_splitter.split_text(final_string)
            new_texts = []
            for t in a:
                t = t.replace('\n\n', ' ')
                ### remove the irrelevant citations and references
                result = re.sub(pattern, "", t)
                result = result.replace(' , ', ', ')
                new_texts.append(result)
            # doc_func = lambda x: x.page_content
            # docs = list(map(doc_func, a))
            chunked_string = ' '.join(new_texts)

        if chunked_string:
            result = model(chunked_string, num_sentences=50, max_length=256)
            summarized_sections = ''.join(result)
        if summarized_sections:
            final_selected_string = final_selected_string + '\n' + summarized_sections
            new_selected_elife_article_val.append(final_selected_string)
        else:
            new_selected_elife_article_val.append(final_selected_string)
            
print(len(new_selected_elife_article_val))

output_file = "/home/zhiweny2/chatbotai/jerome/BioNLP/extractive_aug_dataset/elife_train.json"
with open(output_file, "w") as file:
    json.dump(new_selected_elife_article_train, file)
    
output_file_1 = "/home/zhiweny2/chatbotai/jerome/BioNLP/extractive_aug_dataset/elife_val.json"
with open(output_file_1, "w") as file:
    json.dump(new_selected_elife_article_val, file)

In [1]:

from transformers import(
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
)
from tqdm import tqdm
from datasets import Dataset
import os
import json
from sentence_transformers import SentenceTransformer, util
import torch
import csv
import math
import random
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
import torch
import re
from rouge import Rouge
from transformers import AutoTokenizer
from summarizer.sbert import SBertSummarizer
from summarizer import Summarizer
from transformers import *
import nltk
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader
import logging
from sentence_transformers import SentenceTransformer, util
import torch
import csv
import re
from langchain_text_splitters import NLTKTextSplitter

import random
import numpy as np
# set random seed
random_seed = 42
random.seed(random_seed)

def load_data(dataset, datatype):

    data_folder = '/home/zhiweny2/chatbotai/jerome/BioNLP/biolaysumm2024_data'
    data_path = os.path.join(data_folder, f'{dataset}_{datatype}.jsonl')
    lay_sum = []
    article =[]
    keyword = []
    headings = []
    id = []
    file = open(data_path, 'r')
    for line in (file.readlines()):
        dic = json.loads(line)
        article.append(dic['article'])
        keyword.append(dic['keywords'])
        headings.append(dic['headings'])
        id.append(dic['id'])
        lay_sum.append(dic['lay_summary'])
    
    return article, lay_sum, keyword, headings, id

def load_test_data(dataset, datatype):

    data_folder = '/home/zhiweny2/chatbotai/jerome/BioNLP/biolaysumm2024_data'
    
    data_path = os.path.join(data_folder, f'{dataset}_{datatype}.jsonl')
    article =[]
    keyword = []
    headings = []
    id = []
    file = open(data_path, 'r')
    for line in (file.readlines()):
        dic = json.loads(line)
        article.append(dic['article'])
        keyword.append(dic['keywords'])
        headings.append(dic['headings'])
        id.append(dic['id'])
    
    return article, keyword, headings, id

background = []
objective = []
methods = []
results = []
conclusions = []

with open('/home/zhiweny2/chatbotai/jerome/BioNLP/Structured-Abstracts-Labels-102615.txt', 'r') as file:
    for line in file:
        components = line.strip().split('|')
        title, category, _, _ = components
        if category == 'BACKGROUND':
            background.append(title)
        elif category == 'OBJECTIVE':
            objective.append(title)
        elif category == 'METHODS':
            methods.append(title)
        elif category == 'RESULTS':
            results.append(title)
        elif category == 'CONCLUSIONS':
            conclusions.append(title)
            
background = [item.lower() for item in background]
objective = [item.lower() for item in objective]
methods = [item.lower() for item in methods]
results = [item.lower() for item in results]
conclusions = [item.lower() for item in conclusions]

### PLOS
# plos_article_train, plos_lay_sum_train, plos_keyword_train, plos_headings_train, plos_id_train = load_data('PLOS', 'train')
# # val
# plos_article_val, plos_lay_sum_val, plos_keyword_val, plos_headings_val, plos_id_val = load_data('PLOS', 'val')

elife_article_train, elife_lay_sum_train, elife_keyword_train, elife_headings_train, elife_id_train = load_data('eLife', 'train')
# val
elife_article_val, elife_lay_sum_val, elife_keyword_val, elife_headings_val, elife_id_val = load_data('eLife', 'val')


# plos_article_val = plos_article_val[:2]
# plos_lay_sum_val = plos_lay_sum_val[:2]
# plos_headings_val = plos_headings_val[:2]
# plos_id_val = plos_id_val[:2]
### chunk articles and lay summs

### chunk articles and lay summs for PLOS 

# pattern = r'\s\[.*?\]'
text_splitter = NLTKTextSplitter(chunk_size=600)

### val
new_plos_article_val = []
for s in elife_article_val:
    new_s = s.replace(' . ', '. ')
    new_s = new_s.replace(' , ', ', ')
    new_plos_article_val.append(new_s)
print(len(new_plos_article_val))

### val lay sum
new_plos_lay_sum_val = []
for s in elife_lay_sum_val:
    new_s = s.replace(' . ', '. ')
    new_s = new_s.replace(' , ', ', ')
    new_plos_lay_sum_val.append(new_s)

custom_config = AutoConfig.from_pretrained('/home/zhiweny2/chatbotai/jerome/BioNLP/elife_sentence_level_embedding_model')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('/home/zhiweny2/chatbotai/jerome/BioNLP/elife_sentence_level_embedding_model')
custom_model = AutoModel.from_pretrained('/home/zhiweny2/chatbotai/jerome/BioNLP/elife_sentence_level_embedding_model', config=custom_config)
model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)


non_selected_sec_val = []
selected_sec_val = []
for lay, article, headings in zip(elife_lay_sum_val, elife_article_val, elife_headings_val):
# for lay, article, headings in zip(elife_lay_sum_val, elife_article_val, elife_headings_val):
    sections = article.split('\n')
    temp_selected_sections = []
    temp_sections = []
    for i, (heading, section) in enumerate(zip(headings, sections)):
        heading = heading.lower()
        if heading in background:
            temp_selected_sections.append(section)
        elif heading in methods:
            temp_sections.append(section)
        elif heading in conclusions:
            temp_selected_sections.append(section)
        elif heading in results:
            temp_sections.append(section)
        elif heading in 'abstract':
            temp_selected_sections.append(section)
        else:
            temp_sections.append(section)
            # continue
        
    final_string = ''.join(temp_sections)
    final_selected_string = ''.join(temp_selected_sections)
    non_selected_sec_val.append(final_string)
    selected_sec_val.append(final_selected_string)
    
print("non selected sections val")
print(len(non_selected_sec_val))
print(len(selected_sec_val))

chunked_plos_article_val = []
for article in non_selected_sec_val:
    texts = text_splitter.split_text(article)
    new_texts = []
    for t in texts:
        t = t.replace('\n\n', ' ')
        ### remove the irrelevant citations and references
        # result = re.sub(pattern, "", t)
        result = t.replace(' , ', ', ')
        new_texts.append(result)
    chunked_string = ' '.join(new_texts)
    chunked_plos_article_val.append(chunked_string)
    
print("chunked plos article val")
print(len(chunked_plos_article_val))


### val
final_elife_lay_sum_val = []
new_selected_elife_article_val = []
extractive_sum_val = []
for lay, article, select in zip(new_plos_lay_sum_val, chunked_plos_article_val, selected_sec_val):
    summarized_sections = ""
    if select:
        final_elife_lay_sum_val.append(lay)
        result = model(article, num_sentences=30, max_length=256)
        summarized_sections = ''.join(result)
        extractive_sum_val.append(summarized_sections)
        # new_select = re.sub(pattern, '', select)
        new_select = select.replace(' , ', ', ')
        new_select = new_select + '\n' + summarized_sections
        new_selected_elife_article_val.append(new_select)
    else:
        new_selected_elife_article_val.append(select)
        
print(len(new_selected_elife_article_val))
print(len(extractive_sum_val))





/home/zhiweny2/utils/miniconda3/envs/multi-tagger/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zhiweny2/utils/miniconda3/envs/multi-tagger/lib/python3.8/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/home/zhiweny2/utils/miniconda3/envs/multi-tagger/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
loading configuration file /home/zhiweny2/chatbotai/jerome/BioNLP/elife_sentence_leve

241


All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at /home/zhiweny2/chatbotai/jerome/BioNLP/elife_sentence_level_embedding_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.
Created a chunk of size 788, which is longer than the specified 600
Created a chunk of size 727, which is longer than the specified 600
Created a chunk of size 603, which is longer than the specified 600
Created a chunk of size 801, which is longer than the specified 600
Created a chunk of size 684, which is longer than the specified 600
Created a chunk of size 685, which is longer than the specified 600


non selected sections val
241
241


Created a chunk of size 917, which is longer than the specified 600
Created a chunk of size 737, which is longer than the specified 600
Created a chunk of size 607, which is longer than the specified 600
Created a chunk of size 1208, which is longer than the specified 600
Created a chunk of size 778, which is longer than the specified 600
Created a chunk of size 994, which is longer than the specified 600
Created a chunk of size 1133, which is longer than the specified 600
Created a chunk of size 1081, which is longer than the specified 600
Created a chunk of size 1377, which is longer than the specified 600
Created a chunk of size 683, which is longer than the specified 600
Created a chunk of size 613, which is longer than the specified 600
Created a chunk of size 639, which is longer than the specified 600
Created a chunk of size 696, which is longer than the specified 600
Created a chunk of size 707, which is longer than the specified 600
Created a chunk of size 784, which is longer

chunked plos article val
241


/home/zhiweny2/utils/miniconda3/envs/multi-tagger/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/zhiweny2/utils/miniconda3/envs/multi

241
241


In [2]:
import json
from transformers import AutoTokenizer

# with open('/home/zhiweny2/chatbotai/jerome/BioNLP/extractive_aug_dataset/elife_val_extractive_sum.json', 'r') as f:
#     data = json.load(f)
# print(len(data))

train_list = []
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")
count = 0
for chunk in extractive_sum_val: 
    input_ids = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True).input_ids
    train_list.append(input_ids.shape[1])
    count += input_ids.shape[1]
    # print(input_ids.shape[1])
print(len(train_list))

ave = count / len(extractive_sum_val)
print(ave)

loading configuration file config.json from cache at /home/zhiweny2/.cache/huggingface/hub/models--allenai--led-base-16384/snapshots/38335783885b338d93791936c54bb4be46bebed9/config.json
Model config LEDConfig {
  "_name_or_path": "allenai/led-base-16384",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_window": [
    1024,
    1024,
    1024,
    1024,
    1024,
    1024
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "in

241
846.0829875518672


In [1]:
from transformers import(
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
)
from tqdm import tqdm
from datasets import Dataset
import os
import json
from sentence_transformers import SentenceTransformer, util
import torch
import csv
import math
import random
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
import torch
import re
from rouge import Rouge
from transformers import AutoTokenizer
from summarizer.sbert import SBertSummarizer
from summarizer import Summarizer
from transformers import *
import nltk
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader
import logging
from sentence_transformers import SentenceTransformer, util
import torch
import csv
import re
from langchain_text_splitters import NLTKTextSplitter

import random
import numpy as np
# set random seed
random_seed = 42
random.seed(random_seed)

def load_data(dataset, datatype):

    data_folder = '/home/zhiweny2/chatbotai/jerome/BioNLP/biolaysumm2024_data'
    data_path = os.path.join(data_folder, f'{dataset}_{datatype}.jsonl')
    lay_sum = []
    article =[]
    keyword = []
    headings = []
    id = []
    file = open(data_path, 'r')
    for line in (file.readlines()):
        dic = json.loads(line)
        article.append(dic['article'])
        keyword.append(dic['keywords'])
        headings.append(dic['headings'])
        id.append(dic['id'])
        lay_sum.append(dic['lay_summary'])
    
    return article, lay_sum, keyword, headings, id

def load_test_data(dataset, datatype):

    data_folder = '/home/zhiweny2/chatbotai/jerome/BioNLP/biolaysumm2024_data'
    
    data_path = os.path.join(data_folder, f'{dataset}_{datatype}.jsonl')
    article =[]
    keyword = []
    headings = []
    id = []
    file = open(data_path, 'r')
    for line in (file.readlines()):
        dic = json.loads(line)
        article.append(dic['article'])
        keyword.append(dic['keywords'])
        headings.append(dic['headings'])
        id.append(dic['id'])
    
    return article, keyword, headings, id

background = []
objective = []
methods = []
results = []
conclusions = []

with open('/home/zhiweny2/chatbotai/jerome/BioNLP/Structured-Abstracts-Labels-102615.txt', 'r') as file:
    for line in file:
        components = line.strip().split('|')
        title, category, _, _ = components
        if category == 'BACKGROUND':
            background.append(title)
        elif category == 'OBJECTIVE':
            objective.append(title)
        elif category == 'METHODS':
            methods.append(title)
        elif category == 'RESULTS':
            results.append(title)
        elif category == 'CONCLUSIONS':
            conclusions.append(title)
            
background = [item.lower() for item in background]
objective = [item.lower() for item in objective]
methods = [item.lower() for item in methods]
results = [item.lower() for item in results]
conclusions = [item.lower() for item in conclusions]

print(methods)


### PLOS
# plos_article_train, plos_lay_sum_train, plos_keyword_train, plos_headings_train, plos_id_train = load_data('PLOS', 'train')
# # val
# plos_article_val, plos_lay_sum_val, plos_keyword_val, plos_headings_val, plos_id_val = load_data('PLOS', 'val')

elife_article_train, elife_lay_sum_train, elife_keyword_train, elife_headings_train, elife_id_train = load_data('eLife', 'train')
# val
elife_article_val, elife_lay_sum_val, elife_keyword_val, elife_headings_val, elife_id_val = load_data('eLife', 'val')

print(elife_keyword_val)

/home/zhiweny2/utils/miniconda3/envs/multi-tagger/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zhiweny2/utils/miniconda3/envs/multi-tagger/lib/python3.8/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/home/zhiweny2/utils/miniconda3/envs/multi-tagger/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


['a case report', 'acquisition of evidence', 'action', 'actions', 'activities', 'admission findings', 'ams subject classification', 'analyses', 'analysis', 'analytical approach', 'analytical techniques', 'anatomy', 'anesthesia', 'animal', 'animal or sample population', 'animal population', 'animal studied', 'animal studies', 'animal(s)', 'animals', 'animals and interventions', 'animals and methods', 'animals or sample population', 'animals studied', 'animals, materials and methods', 'animals, methods', 'approach', 'approach and methods', 'area covered', 'area covered in this review', 'areas covered', 'areas covered in the review', 'areas covered in this review', 'article chosen', 'article selection', 'assessment of problem', 'assessment of risk factors', 'assessment tools', 'assessments', 'availability and implementation', 'background & materials and methods', 'background & method', 'background & methods', 'background and design', 'background and method', 'background and methodology', 

In [2]:
wiki_path = '/home/zhiweny2/chatbotai/jerome/BioNLP/wiki/wiki_dict.json'
wiki = json.load(open(wiki_path))

In [33]:
results = []
count = 0
for keywords in elife_keyword_train:
    descriptions = []
    for key in keywords:
        if key in wiki:
            tmp = wiki[key]
            total = ""
            for i in range(len(tmp)):
                if type(tmp[i]) == str:
                    total += key + ' is ' + tmp[i] + '. '
            if total:
                descriptions.append(total)
                count += 1
        else:
            continue
    
    if descriptions:
        final_string = ''.join(descriptions)
        results.append(final_string)
    # final_string = ''.join(descriptions)
    # results.append(final_string)
print(results)
print(len(results))
print(count)

['neuroscience is scientific study of the central nervous system. ', 'ecology is scientific study of the relationships between living organisms. ', 'neuroscience is scientific study of the central nervous system. ', 'cell biology is Scientific Discipline that Studies Cells. ', 'neuroscience is scientific study of the central nervous system. ', 'medicine is field of study for diagnosing, treating and preventing disease. ', 'neuroscience is scientific study of the central nervous system. ', 'neuroscience is scientific study of the central nervous system. ', 'neuroscience is scientific study of the central nervous system. ', 'neuroscience is scientific study of the central nervous system. ', 'ecology is scientific study of the relationships between living organisms. ', 'neuroscience is scientific study of the central nervous system. ', 'neuroscience is scientific study of the central nervous system. ', 'ecology is scientific study of the relationships between living organisms. ', 'evoluti

In [31]:
non_selected_sec_train = []
selected_sec_train = []
for lay, article, headings, keywords in zip(elife_lay_sum_val, elife_article_val, elife_headings_val, elife_keyword_val):
    sections = article.split('\n')
    temp_selected_sections = []
    temp_sections = []
    descriptions = []
    for heading, section in zip(headings, sections):
        heading = heading.lower()
        if heading in background:
            temp_selected_sections.append(section)
        elif heading in methods:
            temp_sections.append(section)
        elif heading in conclusions:
            temp_selected_sections.append(section)
        elif heading in results:
            temp_sections.append(section)
        elif heading in 'abstract':
            temp_selected_sections.append(section)
        else:
            temp_sections.append(section)
            # continue
    
    for keyword in keywords:
        if keyword in wiki:
            tmp = wiki[keyword]
            total = ""
            for i in range(len(tmp)):
                if type(tmp[i]) == str:
                    total += keyword + ' is ' + tmp[i] + '. '
            if total:
                temp_selected_sections.append(total)
        else:
            continue
        
    final_string = ''.join(temp_sections)
    final_selected_string = ''.join(temp_selected_sections)
    non_selected_sec_train.append(final_string)
    selected_sec_train.append(final_selected_string)

158
 Germ cells can be converted into neuronal cells in intact C . elegans upon overexpression of the neuronal transcription factor CHE-1 , simply by depleting the chromatin modifier LIN-53 ( Tursun et al . , 2011; Patel et al . , 2012 ) . This GeCo phenotype can be followed in living animals by monitoring a reporter GFP expressed from the gcy-5 promoter , which otherwise is induced in glutamatergic ASE neurons ( Altun-Gultekin et al . , 2001 ) . In contrast to the spontaneous teratomatous differentiation of meiotic germ cells , observed in the absence of specific RNA-binding proteins ( Ciosk et al . , 2006; Biedermann et al . , 2009; Tocchini et al . , 2014 ) , GeCo is preferentially observed in the pre-meiotic , proliferating germ cells ( Tursun et al . , 2011; Patel et al . , 2012 ) . Consistently , removing the proliferating germ cells , by inhibiting GLP-1Notch signaling , prevents GeCo ( Tursun et al . , 2011 ) . However , because the proliferating germ cells were eliminated , th

In [ ]:
from pyserini.search.lucene import LuceneSearcher

from pyserini.search.faiss import FaissSearcher, TctColBertQueryEncoder

In [ ]:
from pyserini.search.lucene import LuceneSearcher

from pyserini.search.faiss import FaissSearcher, TctColBertQueryEncoder

# wikipedia-dpr-100w
# searcher = LuceneSearcher.from_prebuilt_index('beir-v1.0.0-bioasq-flat')
searcher = LuceneSearcher.from_prebuilt_index('enwiki-paragraphs')

documents = [hit.docid for hit in searcher.search(elife_lay_sum_val[0], k=1)]
documents
# def extract_wiki_search_claims(text, searcher):

#     def process_sent(sent):
#         hits = searcher.search(sent, k=1)
#         if len(hits):
#             return hits[0].raw
#         else:
#             return None

#     with Pool() as pool:
#         sents = nltk.sent_tokenize(text)
#         results = pool.map(process_sent, sents)

#     return [r for r in results if r is not None]